# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to 
the database `'data.sqlite'`.

In [31]:
# Your code here
import pandas as pd
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('data.sqlite')

# Create a cursor object to interact with the database
cursor = conn.cursor()

print("Connected to the database successfully.")

Connected to the database successfully.


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [32]:
# Your code here
query1=""" 
SELECT employees.firstname || ' ' || employees.lastName AS FullName,offices.officecode
FROM employees
JOIN offices ON employees.officecode = offices.officecode
WHERE offices.city = "Boston";
"""
#Execute the query
pd.read_sql(query1,conn)

,FullName,officeCode
0,Julie Firrelli,2
1,Steve Patterson,2


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [33]:
# Your code here
query2="""   
SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) AS numEmployees 
FROM offices
LEFT JOIN employees ON offices.officeCode = employees.officeCode
GROUP BY offices.officeCode,offices.city
HAVING COUNT(employees.employeeNumber) = 0;
"""

#execute the querry
pd.read_sql(query2,conn)

,officeCode,city,numEmployees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [34]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
queryexample=""" 
SELECT 
    o.city AS officeCity,COUNT(c.customerNumber) AS customerCount
FROM offices o
LEFT JOIN employees e ON o.officeCode = e.officeCode
LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY o.city
ORDER BY customerCount DESC;
"""

#execute the query
pd.read_sql(queryexample,conn)

,officeCity,customerCount
0,Paris,29
1,London,17
2,NYC,15
3,San Francisco,12
4,Boston,12
5,Sydney,10
6,Tokyo,5


In [35]:
"""
Question 1: How many customers does each sales representative manage?
"""

# Your code here
query3="""  
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(c.customerNumber) AS numCustomers
FROM employees e
LEFT JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY numCustomers DESC;
"""

#execute the query
pd.read_sql(query3,conn).head()

,employeeNumber,firstName,lastName,numCustomers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1286,Foon Yue,Tseng,7


In [36]:
"""
Question 2: What are the total sales (revenue) for each product?
"""

# Your code here
query4=""" 
SELECT p.productCode, p.productName, SUM(od.quantityOrdered * od.priceEach) AS totalSales
FROM products p
JOIN orderdetails od ON p.productCode = od.productCode
GROUP BY p.productCode, p.productName
ORDER BY totalSales DESC;
"""

#execute the query
pd.read_sql(query4,conn).head()

,productCode,productName,totalSales
0,S18_3232,1992 Ferrari 360 Spider red,276839.98
1,S12_1108,2001 Ferrari Enzo,190755.86
2,S10_1949,1952 Alpine Renault 1300,190017.96
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,170686.00
4,S12_1099,1968 Ford Mustang,161531.48


In [37]:
"""
Question 3:Which customers have placed the most orders?
"""

# Your code here
query5 = """ 
SELECT c.customerNumber, c.customerName, COUNT(o.orderNumber) AS totalOrders
FROM customers c
JOIN orders o ON c.customerNumber = o.customerNumber
GROUP BY c.customerNumber, c.customerName
ORDER BY totalOrders DESC
LIMIT 10;
"""

#execute the query
pd.read_sql(query5,conn)

,customerNumber,customerName,totalOrders
0,141,Euro+ Shopping Channel,26
1,124,Mini Gifts Distributors Ltd.,17
2,114,"Australian Collectors, Co.",5
3,145,Danish Wholesale Imports,5
4,148,"Dragon Souveniers, Ltd.",5
5,323,"Down Under Souveniers, Inc",5
6,353,Reims Collectables,5
7,119,La Rochelle Gifts,4
8,121,Baane Mini Imports,4
9,128,"Blauer See Auto, Co.",4


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [38]:
# Your code here
query6 = """ 
SELECT e.firstName, e.lastName, p.productName
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
ORDER BY e.lastName, e.firstName, p.productName;
"""

#execute the code
pd.read_sql(query6,conn).head()

,firstName,lastName,productName
0,Loui,Bondur,18th century schooner
1,Loui,Bondur,1900s Vintage Bi-Plane
2,Loui,Bondur,1900s Vintage Tri-Plane
3,Loui,Bondur,1903 Ford Model A
4,Loui,Bondur,1903 Ford Model A


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [39]:
# Your code here
query7 = """ 
SELECT e.employeeNumber, e.firstName, e.lastName, SUM(od.quantityOrdered) AS totalProductsSold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY e.lastName, e.firstName;
"""

#Execute the query
pd.read_sql(query7,conn)

,employeeNumber,firstName,lastName,totalProductsSold
0,1337,Loui,Bondur,6186
1,1501,Larry,Bott,8205
2,1401,Pamela,Castillo,9290
3,1188,Julie,Firrelli,4227
4,1611,Andy,Fixter,6246
5,1702,Martin,Gerard,4180
6,1370,Gerard,Hernandez,14231
7,1165,Leslie,Jennings,11854
8,1504,Barry,Jones,7486
9,1612,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [40]:
# Your code here
query8 = """ 
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(DISTINCT od.productCode) AS uniqueProductsSold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING COUNT(DISTINCT od.productCode) > 200
ORDER BY e.lastName, e.firstName;
"""

#execute the query
pd.read_sql(query8,conn)

,employeeNumber,firstName,lastName,uniqueProductsSold


In [41]:
# Query to count the total number of unique products in the 'products' table  
pd.read_sql("""SELECT COUNT(DISTINCT productCode) FROM products;""",conn)


,COUNT(DISTINCT productCode)
0,110


In [42]:
# More than 50 different products

query8 = """ 
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(DISTINCT od.productCode) AS uniqueProductsSold
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
HAVING COUNT(DISTINCT od.productCode) > 50
ORDER BY e.lastName, e.firstName;
"""

#execute the query
pd.read_sql(query8,conn)

,employeeNumber,firstName,lastName,uniqueProductsSold
0,1337,Loui,Bondur,101
1,1501,Larry,Bott,97
2,1401,Pamela,Castillo,100
3,1188,Julie,Firrelli,80
4,1611,Andy,Fixter,82
5,1702,Martin,Gerard,78
6,1370,Gerard,Hernandez,109
7,1165,Leslie,Jennings,107
8,1504,Barry,Jones,98
9,1612,Peter,Marsh,97


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!